In [1]:
!pip install stegano

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.0 MB/s eta 0:00:00


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import hashlib
from stegano import lsb
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend
from google.colab import files
import io

# Geração de chaves RSA para criptografia e descriptografia
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
public_key = private_key.public_key()

# Botões principais para as ações
button_embed = widgets.Button(description="(1) Inserir Texto na Imagem (Esteganografia)", layout=widgets.Layout(width="400px"))
button_retrieve = widgets.Button(description="(2) Extrair Texto da Imagem (Esteganografia)", layout=widgets.Layout(width="400px"))
button_hash = widgets.Button(description="(3) Gerar Hash da Imagem", layout=widgets.Layout(width="400px"))
button_encrypt = widgets.Button(description="(4) Criptografar Mensagem", layout=widgets.Layout(width="400px"))
button_decrypt = widgets.Button(description="(5) Descriptografar Mensagem", layout=widgets.Layout(width="400px"))
button_exit = widgets.Button(description="(S) Sair da Aplicação", layout=widgets.Layout(width="400px"))

# Exibição do menu
display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit)

# Área de saída para resultados e mensagens
display_area = widgets.Output()
message_text = widgets.Text(description="Mensagem:")
encrypted_text_input = widgets.Text(description="Criptografado (hex):")

# Função auxiliar para upload de imagem
def upload_image():
    display_area.clear_output()
    with display_area:
        print("\nPor favor, faça upload de uma imagem.")
    uploaded = files.upload()
    for filename in uploaded.keys():
        image = Image.open(io.BytesIO(uploaded[filename]))
        with display_area:
            print(f"\nImagem '{filename}' enviada com sucesso.")
        return image

# Funções de ação para cada botão
def embed_text_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()

    with display_area:
        original_image = upload_image()
        display(message_text)

    def on_message_submit(change):
        message = message_text.value
        stego_image = lsb.hide(original_image, message)
        stego_filename = "imagem_esteganografada.png"
        stego_image.save(stego_filename)
        display_area.clear_output()
        with display_area:
            print("\nTexto inserido na imagem. Baixe a imagem esteganografada:")
        files.download(stego_filename)

    message_text.on_submit(on_message_submit)

def retrieve_text_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    with display_area:
        stego_image = upload_image()
        message = lsb.reveal(stego_image)
        print(f"\nMensagem extraída: {message}")

def hash_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    with display_area:
        print("\nPor favor, envie uma imagem para gerar seu hash.")
    uploaded = files.upload()
    for filename in uploaded.keys():
        image_data = uploaded[filename]
        image_hash = hashlib.sha256(image_data).hexdigest()
        with display_area:
            print(f"\nHash SHA-256 de '{filename}':\n {image_hash}")

def encrypt_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    display(message_text)

    def on_message_submit(change):
        message = message_text.value
        encrypted_message = public_key.encrypt(
            message.encode(),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        display_area.clear_output()
        with display_area:
            print("\nMensagem criptografada (em hexadecimal):")
            print(encrypted_message.hex())

    message_text.on_submit(on_message_submit)

def decrypt_action(b):
    clear_output(wait=True)
    display(button_embed, button_retrieve, button_hash, button_encrypt, button_decrypt, button_exit, display_area)
    display_area.clear_output()
    display(encrypted_text_input)

    def on_encrypted_submit(change):
        try:
            encrypted_message = bytes.fromhex(encrypted_text_input.value)
            decrypted_message = private_key.decrypt(
                encrypted_message,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            display_area.clear_output()
            with display_area:
                print(f"\nMensagem descriptografada: {decrypted_message.decode()}")
        except ValueError:
            display_area.clear_output()
            with display_area:
                print("\nEntrada inválida para descriptografia. Verifique se a mensagem está em formato hexadecimal.")

    encrypted_text_input.on_submit(on_encrypted_submit)

def exit_action(b):
    clear_output(wait=True)
    display_area.clear_output()
    with display_area:
        print("\nSaindo da aplicação.")

# Conectando as funções aos botões
button_embed.on_click(embed_text_action)
button_retrieve.on_click(retrieve_text_action)
button_hash.on_click(hash_action)
button_encrypt.on_click(encrypt_action)
button_decrypt.on_click(decrypt_action)
button_exit.on_click(exit_action)

# Exibição da área de saída para feedback
display(display_area)


Button(description='(1) Inserir Texto na Imagem (Esteganografia)', layout=Layout(width='400px'), style=ButtonS…

Button(description='(2) Extrair Texto da Imagem (Esteganografia)', layout=Layout(width='400px'), style=ButtonS…

Button(description='(3) Gerar Hash da Imagem', layout=Layout(width='400px'), style=ButtonStyle())

Button(description='(4) Criptografar Mensagem', layout=Layout(width='400px'), style=ButtonStyle())

Button(description='(5) Descriptografar Mensagem', layout=Layout(width='400px'), style=ButtonStyle())

Button(description='(S) Sair da Aplicação', layout=Layout(width='400px'), style=ButtonStyle())

Output()